# Objetivo del proceso a realizar				
## Objetivos del desafío 1			
- Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.		
- Realizar un análisis descriptivo de las principales variables.		
- Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.		

### 1) Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [2]:
path = '~/Desktop/Data/properatti.csv'
data = pd.read_csv(path, sep = ",")

In [3]:
df = data.copy()

### 2) Analisis General del Data Set

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  price_aprox_lo

Revisamos el porcentaje de nulos

Transformamos a category las variables currency state_name country_name place_with_parent_names place_name property_type operation.

Transformamos a string las variables title description properati_url


In [5]:
df.iloc[:,1:7]=df.iloc[:,1:7].astype('category')
df.iloc[:,22:25]=df.iloc[:,22:25].astype(str)
df['currency']=df.currency.astype('category')

In [6]:
df.dtypes

Unnamed: 0                       int64
operation                     category
property_type                 category
place_name                    category
place_with_parent_names       category
country_name                  category
state_name                    category
geonames_id                    float64
lat-lon                         object
lat                            float64
lon                            float64
price                          float64
currency                      category
price_aprox_local_currency     float64
price_aprox_usd                float64
surface_total_in_m2            float64
surface_covered_in_m2          float64
price_usd_per_m2               float64
price_per_m2                   float64
floor                          float64
rooms                          float64
expenses                       float64
properati_url                   object
description                     object
title                           object
image_thumbnail          

### 3) Analisis de duplicados

Vamos a ver si tenemos inmuebles duplicados en nuetro dataset.

Definimos que tenemos un duplicado segun la columna 'properati_url', ya que 2 publicaciones distintas, no pueden remitir a la misma URL de properati.

In [7]:
df.duplicated('properati_url').any()

False

No hay filas duplicadas

Verificamos que todas las propiedades provistas estan para la venta, y ninguna en renta, con lo cual la columna 'operation' no nos aporta mucha informacion.

In [8]:
df.operation.value_counts()

sell    121220
Name: operation, dtype: int64

Vamos a revisar los valores estadisticos de las variables numericas de nuestro Data set

In [9]:
cols=['price','price_aprox_local_currency','price_aprox_usd','surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2','price_per_m2']
df[cols].describe()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2
count,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04
mean,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03
std,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00
25%,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03
50%,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03
75%,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03
max,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06


Aca podemos observar algunos valores extraños en el data set:

* El valor minimo de superficie total, es 0.

* El valor minimo de venta es 0.

* El valor maximo de superficie llega a 20 hectareas


### 4) Analisis de columnas

**Analisis Descriptivo de las principales variables:**

Podemos separar a nuestras variables en 3 grandes grupos por su tipo:

Variables georeferenciales:

* geonames_id                  
* lat-lon                       
* lat                           
* lon                           

Variables relacionadas con el precio del inmueble: 

* price_per_m2: Es nuestra variable objetivo, es la que vamos a querer predecir en la siguiente etapa del trabajo, nos representa el precio por metro cuadrado del inmueble.VARIABLE CONTINUA

* price_usd_per_m2: Esta variable nos indica el valor en dolares por metro cuadrado del inmueble publicado.VARIABLE CONTINUA

* price: Esta variable nos indica el precio del inmueble expresado en la moneda "currency".VARIABLE CONTINUA

* currency: Esta variable nos indica la moneda en la que se publico el inmueble, puede valer (usd,ars,uyu,pen).VARIABLE CATEGORICA

Variables relacionadas con el tamaño del inmueble:

* surface_total_in_m2: Esta variable nos indica los metros cuadrados totales de la propiedad publicada.VARIABLE CONTINUA

* surface_covered_in_m2: Esta variable nos indica los metros cuadrados cubiertos de la propiedad publicada.VARIABLE CONTINUA

Variables que nos dan informacion sobre el lugar donde se encuentra la propiedad

* property_type: Nos indica el tipo de propiedad publicada (ph, apartment, house, store).VARIABLE CATEGORICA

* place_name: Nos indica el barrio(si es dentro de CABA) o localidad, de donde esta la propiedad publicada.VARIABLE CATEGORICA

* state_name: Nos indica la provincia en donde se encuentra la propiedad publicada.VARIABLE CATEGORICA

* place_with_parent_names: Es una combinacion de place_name y state_name, en algunos casos, con algunos datos mas sobre la ubicacion, como el partido. place_with_parent_names = |country_name|state_name|place_name|.VARIABLE CATEGORICA

Variables relacionadas con caracteristicas del inmueble:

* floor: Numero de piso en donde esta ubicado el inmueble(en caso de corresponder).VARIABLE CUALITATIVA ORDINAL

* rooms: Esta variable nos indica la cantidad de ambientes que posee el inmueble.VARIABLE CATEGORICA

* expenses: Valor de las expensas del inmueble (en caso de corresponder).VARIABLE CONTINUA

* description: Una breve descripcion sobre la propiedad en venta.

### 5) Análisis inicial de la distribución de los datos

In [10]:
#Para evitar problemas con los strings, convertimos todo el df en minuscula
df = df.applymap(lambda x: x.lower() if type(x) is str else x)

Droppeamos valores ilogicos, por ejemplo que un valor de superficie total sea menor que el de superficie cubierta

In [15]:
sup_total_incorrecta = df['surface_total_in_m2'] < df['surface_covered_in_m2']

df_sup_total_incorrecta=df[sup_total_incorrecta]
df_sup_total_incorrecta.shape

(1106, 26)

In [16]:
df.drop(index=df_sup_total_incorrecta.index, inplace=True)

In [17]:
(df['surface_total_in_m2'] < df['surface_covered_in_m2']).any()

False

Vamos a corroborar que el precio price_usd_per_m2, coincide con price/surface_covered_in_m2, cuando currency ==usd, para esto nos vamos a crear una nueva columna llamada PRECIO USD POR M2

In [18]:
df_dolares = df[df.currency=='usd']

df['PRECIO USD POR M2']=df_dolares[['price','surface_total_in_m2']].apply(lambda x: 0 if x.surface_total_in_m2 == 0 else x.price/x.surface_total_in_m2, axis=1)

(df['PRECIO USD POR M2']==df['price_usd_per_m2']).all()

False

In [19]:
df['PRECIO USD POR M2'].notnull().sum()

59431

In [20]:
df['price_usd_per_m2'].notnull().sum()

67640

Como PRECIO USD POR M2 es igual que price_usd_per_m2, vamos a imputar los valores que podamos de PRECIO USD POR M2 con price_usd_per_m2

In [23]:
df[['price_usd_per_m2','PRECIO USD POR M2']].sample(5)

,price_usd_per_m2,PRECIO USD POR M2
96374,NaN,NaN
30202,NaN,NaN
93331,NaN,NaN
51088,342.495725,NaN
4664,7172.774869,7172.774869


In [24]:
M2_USD_NULL = df['PRECIO USD POR M2'].isnull()

df.loc[M2_USD_NULL,'PRECIO USD POR M2']=df.loc[M2_USD_NULL,'price_usd_per_m2']

In [ ]:
### Ver si puedo conseguir mas valores para 'PRECIO USD POR M2' de alguna otra columna

In [29]:
df[cols_utiles].head(1)

,property_type,place_name,place_with_parent_names,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description
0,ph,mataderos,|argentina|capital federal|mataderos|,capital federal,62000.0,usd,1093959.0,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,NaN,"2 ambientes tipo casa planta baja por pasillo,..."


In [ ]:
### Ver si puedo conseguir mas valores para 'PRECIO USD POR M2' de alguna otra columna

Separamos las columnas que consideramos tengan alguna relacion con el precio_per_m2 o nos puedan brindar informacion util 

In [28]:
cols_utiles=['property_type','place_name','place_with_parent_names','state_name','price','currency'\
            ,'price_aprox_local_currency','price_aprox_usd','surface_total_in_m2','surface_covered_in_m2'\
            ,'price_usd_per_m2','price_per_m2','floor','rooms','expenses','description', 'PRECIO USD POR M2']

cols_utiles_sin_price=['property_type','place_name','place_with_parent_names','state_name','currency'\
            ,'price_aprox_local_currency','price_aprox_usd','surface_total_in_m2','surface_covered_in_m2'\
            ,'price_usd_per_m2','price_per_m2','floor','rooms','expenses','description', 'PRECIO USD POR M2']

In [ ]:
groupby_property_type=df.groupby('property_type')[cols_utiles_sin_price].median()
groupby_property_type

Dado que 'price', depende del valor de currency, si esta en usd o en ars, vamos a calcularlos por separados, para no crear un bias.

In [ ]:
groupby_property_type_price_usd=df[df.currency=='usd'].groupby('property_type')['price'].median()
groupby_property_type_price_ars=df[df.currency=='ars'].groupby('property_type')['price'].median()

groupby_property_type['price_usd']=groupby_property_type_price_usd
groupby_property_type['price_ars']=groupby_property_type_price_ars
groupby_property_type['cant de propiedades']=df.groupby('property_type')['operation'].count()

groupby_property_type

Al agrupar las propiedades por su tipo, podemos realizar las siguientes observaciones:

*  Las casas son las que suelen tener un mayor precio, pero tambien son las que tienen mas metros cuadrados tienen y pagan mas expensas.

*  Los departamentos son las propiedades que mayor valor por metro cuadrado tienen.

*  Los ph, son la opcion mas economica de propiedad, ya que tienen el precio mas bajo y el menor valor de expensas.

*  La mayoria de los inmuebles publicados son departamentos.

In [ ]:
groupby_provincia_price_usd=df[df.currency=='usd'].groupby('state_name')['price'].median()
groupby_provincia_price_ars=df[df.currency=='ars'].groupby('state_name')['price'].median()

groupby_provincia=df.groupby('state_name')[cols_utiles_sin_price].median()

groupby_provincia['price_usd']=groupby_provincia_price_usd
groupby_provincia['price_ars']=groupby_provincia_price_ars
groupby_provincia['cant de propiedades']=df.groupby('state_name')[cols_utiles_sin_price].count().property_type


groupby_provincia.sort_values('cant de propiedades',ascending=False)

Al agrupar las propiedades por provincia, podemos realizar las siguientes observaciones:

*  Las propiedades que se encuentran en capital federal, son las que tienen un precio por metro cuadrado en usd mas alto.

*  Las propiedades que se encuentran en jujuy, son las que tienen mayor cantidad de metros cuadrados.

*  La mayor cantidad de propiedades en venta, se encuentran en capital federal.


In [ ]:
#GRAFICO

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import output_notebook
from bokeh.models import Legend
from bokeh.resources import INLINE
output_notebook(INLINE)
from bokeh.palettes import Set2

In [ ]:
colors = Set2[5]


In [ ]:
len(df.state_name.unique())

In [ ]:
xrange=list(df.state_name.unique()) 

offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
provincia_offset = list(zip(xrange, np.array(offsets)*2))

r0=list(groupby_provincia.price_per_m2)
r1=list(groupby_provincia.price_usd_per_m2)

In [ ]:
g = figure(x_range=xrange, plot_width=1000, plot_height=600
           , title='PRECIO POR METRO CUADRADO POR PROVINCIA', x_axis_label='PROVINCIAS',y_axis_label='$/m2')


r0 = g.vbar(x=xrange,           top=r0, width=0.5, bottom=0, color=colors[0])
r1 = g.vbar(x=provincia_offset, top=r1, width=0.5, bottom=0, color=colors[1])

show(g)

### 6) Analisis de consistencia y calidad de los datos

#### Los precios estan en distintas monedas, vamos a llevar todo a usd.

In [ ]:
df.currency.value_counts()

##### Primero, vamos a estudiar los casos donde currency vale pen y uyu

In [ ]:
df_currency_uyu=df[df.currency=='uyu']
list(df_currency_uyu.description)

In [ ]:
df_currency_uyu

In [ ]:
#Entonces vamos a imputarle el valor de price = $ 650.000.000, con currency='ars'
df.loc[107390,'price']=650000000
df.loc[107390,'currency']='ars'
df.loc[107390,['price','currency']]

In [ ]:
list(df[df.currency=='pen'].description)

La descripcion que podemos encontrar en ambos casos, son un poco confusas sobre los precios que usariamos para imputar, por lo que optamos por dropear estas 2 filas.

In [ ]:
df[df.currency=='pen']

In [ ]:
df.drop(index = [50387,50388], inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
price_per_m2:
    currency=usd
        price_per_m2 = price/surface_covered_in_m2
        Si coinciden surface_covered_in_m2 y surface_total_in_m2, entonces price_per_m2 == price_usd_per_m2
    currency=ars
        price_per_m2=price/surface_covered_in_m2(apartment and house), hay que chequear con store
        Si coinciden surface_covered_in_m2 y surface_total_in_m2, entonces price_usd_per_m2 == 17.830501*price_per_m2
    currency=uyu
        segun la descripcion, hay q ponerle price=650.000.000 y currency=ars
    currency=pen
        Segun la descripcion es un departamento de pozo, con cochera, lo vamos a tomar $956.000, currency=ars, 
        tomamos la superficie que nos dice como surface_covered_in_m2=47m2.

### Floor

Sabemos que los departamentos que se encuentran en los pisos superiores de los edificios, suelen tener precios superiores a los que se encuentran cerca de la planta baja, pero nuestra columna floor, esta muy vacia, vamos a ver si podemos completarla con la informacion que encontramos en la columna 'description'

In [ ]:
df.property_type.value_counts()

In [ ]:
departamento_mask = df.property_type=='apartment'
patron_piso = re.compile(r'(\d+(º)?(\s)?(piso))|((piso)(\s)?\d+(º)?)')

encuentra_piso=df.loc[departamento_mask,'description'].apply(lambda x: re.search(patron_piso,str(x)))

encuentra_piso_mask=encuentra_piso.notnull()

pisos=encuentra_piso[encuentra_piso_mask].apply(lambda x: x.group())


In [ ]:
# pisos.shape
pisos.sample(10)

Nuestro REGEX matchea dentro de todo bastante bien, solo debemos filtrar algunos valores altos. Pero primero, vamos a filtrar y convertir a int, nuestra variable pisos

In [ ]:
pisos_sin_piso=pisos.replace("piso"," ",regex=True)
pisos_sin_piso = pisos_sin_piso.apply(lambda x: str(x).strip())
pisos_clean_str = pisos_sin_piso.replace("º","",regex=True)
pisos_int = pisos_clean_str.apply(lambda y: int(y))
pisos_int.sample(5)

Ahora, voy a intentar sacarme de encima a los pocos valores que el regex me tomo por error, primero defino con mi criterio, que los edificios no pueden tener mas de 40 pisos

In [ ]:
pisos_filtrado_mask = pisos_int <= 50

pisos_filtrado_menor50=pisos_int[pisos_filtrado_mask]

In [ ]:
g=sns.histplot(pisos_filtrado_menor50)
sns.set(rc = {'figure.figsize':(15,6)})
g.set_title("DISTRIBUCION DE LOS DEPARTAMENTOS SEGUN LOS PISOS", fontsize = 17)
g.set_xlabel("PISO DE LA PROPIEDAD", fontsize = 15)
g.set_ylabel("CANTIDAD DE DEPARTAMENTOS", fontsize = 15)

La gran mayoria de los departamentos, rondan entre el 2do y 10mo piso

In [ ]:
pisos_filtrado_menor50.median()

Porcentaje de valores faltantes en floor para viviendas tipo departamento

In [ ]:
df.floor[departamento_mask].isnull().sum()/df.floor[departamento_mask].shape[0]*100

Ahora con estos valores de pisos, voy a imputar mi columna floor, en las filas donde property_type sea apartment y floor tenga valor nan

In [ ]:
null_floor_mask = df.floor.isnull()
departamento_mask
departamento_y_floor_null=np.logical_and(departamento_mask,null_floor_mask)

df.loc[departamento_y_floor_null,'floor']=pisos_filtrado_menor50

In [ ]:
df.floor[departamento_mask].isnull().sum()/df.floor[departamento_mask].shape[0]*100

Bajo el porcentaje de nulos, pero aun sigue siendo muy alto para tomar esta columna como relevante.

### Rooms

Es posible que la cantidad de ambientes en los departamentos, que se encuentren en la misma zona (place_name), influya en el valor del inmueble, pero tenemos un 60.9% de columnas Null, veremos si podemos obtener alguna informacion de la columna description, como hicimos con floor.

In [ ]:
patron_ambientes = re.compile(r'\d+(\s)?(ambiente(s)?)|\d+(\s)?(amb(\.)?)')


encuentra_ambientes=df.loc[departamento_mask,'description'].apply(lambda x: re.search(patron_ambientes,str(x)))

encuentra_ambientes_mask=encuentra_ambientes.notnull()

ambientes=encuentra_ambientes[encuentra_ambientes_mask].apply(lambda x: x.group())
ambientes.sample(30)
# ambientes.shape
ambientes.value_counts()

Podemos ver que nuestro Regex funciono bastante bien, pero hay alguno valores que hay que filtrar, para esto primero vamos a convertir a dato numerico 

In [ ]:
ambientes_sin_ambientes=ambientes.replace("ambientes"," ",regex=True)
ambientes_sin_ambientes_sin_ambiente=ambientes_sin_ambientes.replace("ambiente"," ",regex=True)
ambientes_sin_ambientes_sin_ambiente_sin_amb=ambientes_sin_ambientes_sin_ambiente.replace("amb(\.)?"," ",regex=True)

ambientes_clean_str = ambientes_sin_ambientes_sin_ambiente_sin_amb.apply(lambda x: str(x).strip())
ambientes_int = ambientes_clean_str.apply(lambda y: int(y))
ambientes_int.sample(5)

Voy a tomar un criterio de que mas de 10 ambientes, en un departamento, es un valor incorrecto

In [ ]:
ambientes_menor_10=ambientes_int<10
ambientes_clean=ambientes_int[ambientes_menor_10]

In [ ]:
g1=sns.histplot(ambientes_clean)
sns.set(rc = {'figure.figsize':(15,6)})
g1.set_title("CANTIDAD DE DEPARTAMENTOS EN FUNCION DE SU N DE AMBIENTES", fontsize = 17)
g1.set_xlabel("Nº AMBIENTES", fontsize = 15)
g1.set_ylabel("CANTIDAD DE DEPARTAMENTOS", fontsize = 15)

Podemos observar que la gran mayoria de los departamentos en venta, tienen 2 y 3 ambientes

Porcentaje de valores faltantes en la columna rooms, para viviendas tipo departamento.

In [ ]:
df.rooms[departamento_mask].isnull().sum()/df.rooms[departamento_mask].shape[0]*100

Ahora vamos a imputar con ambientes_clean, algunos de los valores faltantes de rooms

In [ ]:
rooms_null_mask=df.rooms.isnull()
departamento_rooms_null_mask = np.logical_and(rooms_null_mask,departamento_mask)

df.loc[departamento_rooms_null_mask,'rooms']=ambientes_clean

df.rooms[departamento_mask].isnull().sum()/df.rooms[departamento_mask].shape[0]*100

Podemos ver como bajamos casi un 20% el valor de NaN rows en los inmuebles tipo departamento

### Cocheras

Vamos a filtrar los departamentos que tengan cochera, para estudiar si el precio por m2 aumenta en estos casos.

In [ ]:
df_departamentos = df[departamento_mask]

patron_cochera = re.compile(r'cochera(s)?|garage(s)?|garaje(s)?')

encuentra_cochera=df.loc[departamento_mask,'description'].apply(lambda x: re.search(patron_cochera,str(x)))

departamentos_con_cochera_mask=encuentra_cochera.notnull()


departamentos_con_cochera = df_departamentos[departamentos_con_cochera_mask]
departamentos_con_cochera.sample(3)

### Expenses

## Piscinas

Vamos a ver de todas las propiedades, cuales tienen pileta, para ver si esto afecta al valor por metro cuadrado de la propiedad

In [ ]:
patron_pileta = re.compile(r'piscina|pileta')

encuentra_pileta=df['description'].apply(lambda x: re.search(patron_pileta,str(x)))

inmuebles_con_pileta_mask=encuentra_pileta.notnull()


inmuebles_con_pileta = df[inmuebles_con_pileta_mask]

In [ ]:
list(inmuebles_con_pileta.description.sample(1))

## Filtrado de columnas con demasiados elementos faltantes, que puedan producirnos un bias muy alto si queremos imputarlas de alguna manera

Vamos a quedarnos con aquellas columnas que por lo menos tengan el 60% de los valores sin NaN.

In [ ]:
threshold= df.shape[0]*0.6
df_cols_utiles = df[cols_utiles].dropna(thresh = threshold ,axis=1)

In [ ]:
df_cols_utiles.shape[1]

In [ ]:
len(cols_utiles)

De esta manera, dropeamos las columnas 'rooms', 'floor', 'expenses', 'price_usd_per_m2'

In [ ]:
df_cols_utiles.isnull().sum()/df_cols_utiles.shape[0]*100

Dado que 'price_usd_per_m2', está muy cerca del umbral, veremos si tiene alguna utilidad con nuestra variable mas relevante o podemos descartarlo directamente.

In [ ]:
notnull_price_usd_per_m2_mask=df['price_usd_per_m2'].notnull()
null_price_per_m2_mask = df['price_per_m2'].isnull()

(notnull_price_usd_per_m2_mask==null_price_per_m2_mask).any()
#Hay alguna fila donde price_usd_per_m2, tenga un valor util y price_per_m2, sea NaN?

Debido a la proximidad con el umbral que fijamos, y que puede aportarnos informacion util, vamos a cambiar el umbral al 55% de valores utiles, para no perder 'price_usd_per_m2'

In [ ]:
threshold2= df.shape[0]*0.55
df_cols_utiles2 = df[cols_utiles].dropna(thresh = threshold2 ,axis=1)

In [ ]:
df_cols_utiles2.isnull().sum()/df_cols_utiles2.shape[0]*100

In [ ]:
df_cols_utiles2.sample(5)

**Ahora vamos a comprobar si columnas como'price_aprox_local_currency','price_aprox_usd','place_with_parent_names','currency', tienen alguna utilidad para nosotros, o la informacion que nos dan, las tenemos contenidas en otras columnas**

## 'price_aprox_local_currency','price_aprox_usd',

In [ ]:
df_cols_utiles2[['price_aprox_local_currency','price_aprox_usd','price','currency']].sample(5)

Observando el dataset, vemos que price_aprox_local_currency(ars), se aproxima al valor de price, cuando currency esta en 'ars'

Tambien vemos que price_aprox_usd, coincide con el valor de price, cuando currency esta en 'usd'.


In [ ]:
ars_mask = df_cols_utiles2.currency == 'ars'

df_cols_utiles2_ars=df_cols_utiles2[ars_mask]

rel = df_cols_utiles2_ars.price/df_cols_utiles2_ars.price_aprox_local_currency
relacion = rel.mean()
relacion

Observamos que la relacion entre price y price_aprox_local_currency, es "relacion"


Nuestro objetivo, es eliminar price_aprox_local_currency y price_aprox_usd, una vez las usemos para completar price

Veamos primero si nos son utiles estas columnas para lo que queremos hacer.

In [ ]:
notnull_price_aprox_local_currency=df_cols_utiles2.price_aprox_local_currency.notnull()
notnull_price_aprox_usd=df_cols_utiles2.price_aprox_usd.notnull()
price_ars_usd = (df_cols_utiles2.currency == 'usd') | (df_cols_utiles2.currency == 'ars')
null_price_ars_usd=df_cols_utiles2.price[price_ars_usd].isnull()


price_local_o_price_usd=np.logical_or(notnull_price_aprox_local_currency,notnull_price_aprox_usd)

null_price_notnull_price_local_o_price_usd=np.logical_and(price_local_o_price_usd,null_price_ars_usd)

null_price_notnull_price_local_o_price_usd.any()

Por lo tanto, comprobamos que **NO HAY** ningun valor no nulo en price_aprox_usd o price_aprox_local_currency, que podamos utilizar para imputar price, con currency 'ars' o 'usd'

## place_with_parent_names

place_with_parent_names = |country_name|state_name|place_name|, con lo cual, no nos da informacion extra, y es mas facil utilizar para los groupby sus columnas "hijas", con lo cual, la utilizaremos para completar "place_name", que tiene algunos nulos y luego la descartaremos.

Vamos a comprobar si podemos completar place_name, con place_with_parent_names. Para esto, vamos a eliminar las filas donde haya NaN en place_name y comparar si es igual al anteultimo elemento de place_with_parent_names, al aplicarle un split(el ultimo es el pipe).

In [ ]:
place_name_tester=df_cols_utiles2.place_with_parent_names.apply(lambda x: x.split('|')[-2])
place_name_tester

In [ ]:
place_name_null_mask = df_cols_utiles2.place_name.isnull()

In [ ]:
(place_name_tester[~place_name_null_mask]==df_cols_utiles2.place_name[~place_name_null_mask]).all()

Comprobamos que podemos imputar el valor de place_name, a partir de la columna place_with_parent_names, asi que vamos a imputarlo

In [ ]:
df_cols_utiles2.loc[place_name_null_mask,'place_name'] = place_name_tester

df_cols_utiles2.place_name.isnull().sum()

Ahora dropeamos algunas columnas de nuestro data frame

In [ ]:
df_cols_utiles2.head(1)

In [ ]:
df_cols_utiles3=df_cols_utiles2.drop(['place_with_parent_names','price_aprox_local_currency','price_aprox_usd'], axis=1)
df_cols_utiles3.head(3)

Estudiando el dataset, podemos observar, que el valor de price_usd_per_m2, es igual al de price_per_m2, cuando surface_covered_in_m2 es igual a surface_total_in_m2, SIEMPRE QUE CURRENCY ESTE EN USD

In [ ]:
df_cols_utiles2[df_cols_utiles2.surface_covered_in_m2 == df_cols_utiles2.surface_total_in_m2][['price_usd_per_m2','price_per_m2']]

In [ ]:
## IMPUTACION A price_usd_per_m2 con ,price_per_m2 para filas donde currency es 'usd'

In [ ]:
currency_usd_mask=df_cols_utiles2.currency == 'usd'
cubiertos_igual_totales=df_cols_utiles2.surface_covered_in_m2==df_cols_utiles2.surface_total_in_m2

df_price_per_m2_igual_price_usd_per_m2= df_cols_utiles2[np.logical_and(currency_usd_mask,cubiertos_igual_totales)]

df_price_per_m2_igual_price_usd_per_m2.price_per_m2.isnull().any()

Comprobamos que **NO HAY** ningun valor de `price_usd_per_m2` que podamos imputar usando la relacion el valor de `price_per_m2`, es igual al de price_per_m2, cuando surface_covered_in_m2 es igual a surface_total_in_m2, SIEMPRE QUE CURRENCY ESTE EN USD

In [ ]:
df_cols_utiles2.price_usd_per_m2.notnull().sum()

In [ ]:
df_cols_utiles2.price_per_m2.notnull().sum()

In [ ]:
## IMPUTACION A price_per_m2 con price_usd_per_m2, para filas donde currency es 'usd'